### Load

In [2]:
import pickle
import pandas as pd
with open('../models/lasso.pkl', 'rb') as f:
    lasso = pickle.load(f)
with open('../models/scalers/lasso_scaler.pkl', 'rb') as f:
    dt = pickle.load(f)

with open('../models/dt.pkl', 'rb') as f:
    dt = pickle.load(f)
with open('../models/scalers/dt_scaler.pkl', 'rb') as f:
    dt = pickle.load(f)

with open('../models/gbt.pkl', 'rb') as f:
    gbt = pickle.load(f)
with open('../models/scalers/gbt_scaler.pkl', 'rb') as f:
    dt = pickle.load(f)

lasso_df = pd.read_csv('../data/7_model_specific_data_sets/lasso_scaled.csv')
dt_df = pd.read_csv('../data/7_model_specific_data_sets/dt_scaled.csv')
gbt_df = pd.read_csv('../data/7_model_specific_data_sets/gbt_scaled.csv')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score,mean_squared_error, r2_score
def two_d_compare(X_test,y_test,y_pred,model_name):
    area = (12 * np.random.rand(40))**2 
    plt.subplots(ncols=2, figsize=(10,4))
    plt.suptitle('Actual vs Predicted data : ' +model_name + '. Variance score: %.2f' % r2_score(y_test, y_pred))

    plt.subplot(121)
    plt.scatter(X_test, y_test, alpha=0.8, color='#8CCB9B')
    plt.title('Actual')

    plt.subplot(122)
    plt.scatter(X_test, y_pred,alpha=0.8, color='#E5E88B')
    plt.title('Predicted')

    plt.show()

def two_vs_three(x_test,y_test,y_pred,z=None, isLinear = False) : 
    
    area = 60
    

    fig = plt.figure(figsize=(12,6))
    fig.suptitle('2D and 3D view of sales price data')

    # First subplot
    ax = fig.add_subplot(1, 2,1)
    ax.scatter(x_test, y_test, alpha=0.5,color='blue', s= area)
    ax.plot(x_test, y_pred, alpha=0.9,color='red', linewidth=2)
    ax.set_xlabel('YEAR BUILT')
    ax.set_ylabel('SELLING PRICE')
    
    plt.title('YEARBUILT vs SALEPRICE')
    
    if not isLinear : 
    # Second subplot
        ax = fig.add_subplot(1,2,2, projection='3d')

        ax.scatter(z, x_test, y_test, color='blue', marker='o')
        ax.plot(z, x_test, y_pred, alpha=0.9,color='red', linewidth=2)
        ax.set_ylabel('YEAR BUILT')
        ax.set_zlabel('SELLING PRICE')
        ax.set_xlabel('LOT AREA')

    plt.title('LOT AREA vs YEAR BUILT vs SELLING PRICE')

    plt.show()